In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
X = df.drop('label',axis=1)
y= df['label']
X.shape

(18285, 4)

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
voc_size=5000

In [10]:
messages=X.copy()

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
#nltk.download('stopwords')

In [14]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
X.shape

(18285, 4)

In [16]:
X = X[~X['title'].str.isnumeric()]


In [17]:
X.shape

(18285, 4)

In [18]:
print(messages.columns)


Index(['index', 'id', 'title', 'author', 'text'], dtype='object')


In [19]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [34]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1967, 3334, 1292, 4131, 4061, 148, 3654, 1779, 3208, 3340],
 [4716, 942, 4380, 452, 2298, 1993, 4405],
 [1437, 3021, 3821, 3085],
 [1267, 2353, 4089, 114, 4221, 4596],
 [4973, 2298, 2888, 2149, 2933, 4152, 2298, 4285, 1965, 2105],
 [4694,
  3566,
  3085,
  3658,
  4754,
  1527,
  4197,
  3104,
  910,
  4484,
  3740,
  3483,
  3572,
  351,
  4405],
 [3678, 2130, 1598, 4314, 3310, 4313, 1501, 1691, 1927, 2998, 4642],
 [4583, 3013, 3019, 477, 564, 4407, 1527, 385, 1927, 2998, 4642],
 [1377, 1916, 3351, 902, 4935, 2712, 2417, 4679, 1527, 3231],
 [4271, 3696, 2284, 1271, 3323, 4567, 1164, 1690],
 [125, 2284, 2594, 4422, 1942, 1849, 1446, 3828, 788, 2926, 3625],
 [114, 978, 4061, 2712, 1527, 564],
 [3506, 2319, 3925, 2746, 277, 645, 1528, 3291, 2881],
 [4033, 435, 4504, 741, 929, 1938, 2213, 1927, 2998, 4642],
 [1253, 4573, 914, 2643, 4483, 1927, 2998, 4642],
 [89, 338, 4889, 4300, 4035, 1959, 1738, 749, 2117, 2902],
 [1760, 1445, 942],
 [612, 3723, 1293, 2846, 1527, 3357, 1653, 4405],
 [4

One-hot encoding is a simple way to represent words numerically, but it doesn't capture semantic similarities between words.

<h3>Word Embeddings

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1967 3334 1292 ...    0    0    0]
 [4716  942 4380 ...    0    0    0]
 [1437 3021 3821 ...    0    0    0]
 ...
 [ 794 1934 3198 ...    0    0    0]
 [4668  564 4560 ...    0    0    0]
 [ 569 2794  595 ...    0    0    0]]


And Word Embeddings just add zero in the beginning or the End

In [22]:
embedded_vector_features=40
model=Sequential()

In [33]:
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# new_learning_rate = 0.001
# model.compile(optimizer=SGD(learning_rate=new_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
 embedding_1 (Embedding)     (None, 1, 40)             200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 513002 (1.96 MB)
Trainable params: 513002 

In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 36ms/step - loss: 0.3177 - accuracy: 0.8460 - val_loss: 0.1983 - val_accuracy: 0.9160
Epoch 2/10
192/192 [==============================] - 6s 29ms/step - loss: 0.1405 - accuracy: 0.9459 - val_loss: 0.1955 - val_accuracy: 0.9140
Epoch 3/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1014 - accuracy: 0.9629 - val_loss: 0.2517 - val_accuracy: 0.9122
Epoch 4/10
192/192 [==============================] - 5s 29ms/step - loss: 0.0756 - accuracy: 0.9729 - val_loss: 0.2583 - val_accuracy: 0.9138
Epoch 5/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0567 - accuracy: 0.9808 - val_loss: 0.3353 - val_accuracy: 0.9097
Epoch 6/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0401 - accuracy: 0.9874 - val_loss: 0.3662 - val_accuracy: 0.9052
Epoch 7/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0282 - accuracy: 0.9915 - val_loss: 0.3686 - val_accuracy: 0.907

Adding Droupout Layer 

In [28]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 5ms/step


In [30]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,y_pred)

array([[3113,  306],
       [ 246, 2370]], dtype=int64)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



<h3>Additional Tasks to perform on this project
<h5>->In the Neural Network try using different Optimizers along with changing learning rate to see its effect on accuracy
<h5>->Increase the number of epochs and apply a limit to it to get the best accuracy  
<h5>->